In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = "udacity-project-cluster"

try:
    compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(workspace=ws, name=compute_cluster_name, 
        provisioning_configuration=compute_cluster_config)
     compute_target.wait_for_completion(show_output=True)
        
print(compute_target.get_status().serialize())

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
parameter_sampler = RandomParameterSampling( {
        "--C": uniform(0.05, 0.1),
        "--max_iter": choice(16, 32, 64, 128)}) ### YOUR CODE HERE ###

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2, delay_evaluation=5) ### YOUR CODE HERE ###
"""Bandit terminates runs where the primary metric is not within 
the specified slack factor/slack amount compared to the best performing run."""

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory="./", 
    compute_target=compute_target, entry_script="train.py")### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                     hyperparameter_sampling = parameter_sampler,
                                     policy = policy,
                                     primary_metric_name="accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE
                                     max_total_runs = 35,
                                     max_concurrent_runs = 5
                                     )### YOUR CODE HERE ###

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hd_config)
RunDetails(hyperdrive_run).show() 
### YOUR CODE HERE ###

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

best_run.get_file_names() #To get the actual model file

### YOUR CODE HERE ###

In [ ]:
best_run.download_file(name="outputs/model.joblib", output_file_path="./outputs/")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

Url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=Url)

### YOUR CODE HERE ###

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)
    
x_train, x_test, y_train, y_test = train_test_split(x, y)
    
data = pd.concat([x_train,y_train],axis=1)
data.head()

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task= "classification",
    primary_metric= "Accuracy",
    compute_target=compute_target
    training_data= data,
    label_column_name= 'y',
    n_cross_validations= 4)

In [2]:
# Submit your automl run
automl_run = exp.submit(automl_config, show_output = False)
# Printing the Run details 
RunDetails(automl_run).show()
# Ensuring that AutoML run is completed
automl_run.wait_for_completion(show_output=True)
### YOUR CODE HERE ###

In [1]:
# Retrieve and save your best automl model.

best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
joblib.dump(fitted_model, "fitted_automl_model.joblib")
### YOUR CODE HERE ###

NameError: name 'automlrun' is not defined

In [ ]:
cpu_cluster.delete()